In [1]:
#All imports at the top
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
#Adapter Layer - All functions which interact with s3 storage
#Read csv files to convert to df
    #Remove part of for loop into a function
    #Think of changing parts of the code to be as input arguments, creating default values if needed 
    #Adding what needs to be added in or used like a bucket, key, the decoding and sep, changing the code
def read_csv_to_df(bucket,key,decoding='utf-8',sep=','):
    csv_obj = bucket.Object(key=key).get().get('Body').read().decode(decoding)
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=sep)
    return df

#Write df to s3
    #Renaming the specific name to be more general from bucket_target to bucket, and df_all to df
    #Add a return of true
def write_df_to_s3(bucket,df,key):
    out_buffer = BytesIO()
    df.to_parquet(out_buffer,index=False)
    bucket.put_object(Body=out_buffer.getvalue(),Key=key)
    return True

def list_files_in_prefix(bucket, prefix):
    files = [obj.key for obj in bucket.objects.filter(Prefix=prefix)]
    return files


In [3]:
# Application Layer (core) - ETL
def extract(bucket,date_list):
    files = [key for date in date_list for key in list_files_in_prefix(bucket,date)]
    df = pd.concat([read_csv_to_df(bucket,obj) for obj in files], ignore_index=True)
    return df

def transform_report1(df,columns,arg_date):
    df = df.loc[:,columns]
    df.dropna(inplace=True)
    df['opening_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')
    df['closing_price'] = df.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')
    df = df.groupby(['ISIN','Date'],as_index=False).agg(opening_price_eur=('opening_price','min'),closing_price_eur=('closing_price','min'), minimum_price_eur=('MinPrice','min'), maximum_price_eur=('MaxPrice','max'), daily_traded_volume=('TradedVolume','sum'))
    df['prev_closing_price'] = df.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)
    df['change_prev_closing_%'] = (df['closing_price_eur'] - df['prev_closing_price']) / df['prev_closing_price'] * 100
    df.drop(columns=['prev_closing_price'],inplace=True)
    df = df.round(decimals=2)
    df = df[df.Date>=arg_date]
    return df

def load(bucket,df,trg_key,trg_format):
    key = trg_key + datetime.today().strftime("%Y%m%d_%H%M%S") + trg_format  
    write_df_to_s3(bucket,df,key)
    
    #Note there are two buckets in the original function so we have to differentiate by name
def etl_report1(src_bucket,trg_bucket,date_list,columns,arg_date,trg_key,trg_format):
    df = extract(src_bucket,date_list)
    df = transform_report1(df,columns,arg_date)
    load(trg_bucket,df,trg_key,trg_format)
    return True

In [4]:
# Application Layer - not core

# def return_objects(bucket,arg_date,src_format):
#     min_date = datetime.strptime(arg_date,src_format).date() - timedelta(days=1)
#     objects = [obj.key for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= min_date]
#     return objects

#Rewrite the function from above
def return_date_list(bucket,arg_date,src_format):
    min_date = datetime.strptime(arg_date,src_format).date() - timedelta(days=1)
    today = datetime.today().date()
    return_date_list = [(min_date + timedelta(days=x)).strftime(src_format) for x in range(0,(today-min_date).days + 1)]    
    return return_date_list

In [5]:
#main function entry point
def main():
    #Parameters/configurations
    #Later read config
    arg_date = '2022-05-11'
    src_format = '%Y-%m-%d'
    src_bucket = 'xetra-1234'
    trg_bucket = 'xetra-data-etl-destination'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice',
           'EndPrice', 'TradedVolume']
    trg_key = 'xetra_daily_report_'
    trg_format = '.parquet'
    
    #Init
    s3 = boto3.resource('s3')
    #Note to keep the names different
    bucket_src = s3.Bucket(src_bucket)
    bucket_trg = s3.Bucket(trg_bucket)
    
    #run application
    date_list = return_date_list(bucket_src, arg_date, src_format)
    etl_report1(bucket_src,bucket_trg,date_list,columns,arg_date,trg_key,trg_format)

In [6]:
#run
main()

### Reading the uploaded file

In [7]:
trg_bucket = 'xetra-data-etl-destination'
s3 = boto3.resource('s3')
bucket_trg = s3.Bucket(trg_bucket)
    
for obj in bucket_trg.objects.all():
    print(obj.key)

xetra_daily_report_20230422_151559.parquet
xetra_daily_report_20230422_171304.parquet
xetra_daily_report_20230422_211652.parquet
xetra_daily_report_20230422_220232.parquet


In [8]:
prq_obj = bucket_trg.Object(key='xetra_daily_report_20230422_220232.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [9]:
df_report

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_traded_volume,change_prev_closing_%
0,AT000000STR1,2022-05-11,39.15,39.25,38.65,39.60,914,0.77
1,AT000000STR1,2022-05-12,39.15,39.25,38.65,39.60,914,0.00
2,AT000000STR1,2022-05-13,39.15,39.25,38.65,39.60,914,0.00
3,AT000000STR1,2022-05-14,39.30,38.50,38.00,39.30,1299,-1.91
4,AT000000STR1,2022-05-15,38.70,38.80,38.35,38.80,400,0.78
...,...,...,...,...,...,...,...,...
752072,XS2434891219,2022-12-27,3.44,3.50,3.44,3.50,0,0.00
752073,XS2434891219,2022-12-28,3.44,3.66,3.42,3.66,0,4.53
752074,XS2434891219,2022-12-29,3.44,3.66,3.42,3.66,0,0.00
752075,XS2434891219,2022-12-30,3.44,3.66,3.42,3.66,0,0.00
